In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle
import warnings

In [2]:
# load dataset
df = pd.read_csv("./data/cleandata.csv")
df.head()

,DischargeFromED,Foreigners,gender,marital_Status,IsGlobal,risk_chance,age_group,is_peak_hour,is_weekend,support,ResidentDay_log
0,0,0,0.0,2.0,0.0,0,3,0,0,2,2.397895
1,0,0,1.0,2.0,0.0,0,2,0,0,2,1.098612
2,0,0,0.0,2.0,0.0,0,3,0,0,2,0.000000
3,0,0,1.0,2.0,0.0,0,3,0,0,2,2.302585
4,0,0,1.0,2.0,0.0,0,1,0,0,2,0.693147


In [3]:
df = pd.get_dummies(df, drop_first=True)


In [4]:
df.dtypes

DischargeFromED        int64
Foreigners             int64
gender               float64
marital_Status       float64
IsGlobal             float64
risk_chance            int64
is_peak_hour           int64
is_weekend             int64
support                int64
ResidentDay_log      float64
age_group_1             bool
age_group_2             bool
age_group_3             bool
age_group_Unknown       bool
dtype: object

In [5]:
df['age_group_1'] = df['age_group_1'].astype(int)
df['age_group_2'] = df['age_group_2'].astype(int)
df['age_group_3'] = df['age_group_3'].astype(int)
df['age_group_Unknown'] = df['age_group_Unknown'].astype(int)

In [6]:
df.dtypes


DischargeFromED        int64
Foreigners             int64
gender               float64
marital_Status       float64
IsGlobal             float64
risk_chance            int64
is_peak_hour           int64
is_weekend             int64
support                int64
ResidentDay_log      float64
age_group_1            int64
age_group_2            int64
age_group_3            int64
age_group_Unknown      int64
dtype: object

In [7]:
df["DischargeFromED"].isna().sum()

np.int64(0)

Data Train split  and Modeling 

In [8]:
# deprate X and y


X = df.drop('DischargeFromED', axis=1)
y = df['DischargeFromED']


In [9]:
# Identify categorical & numeric columns

cat_cols = [
    'Foreigners', 'gender', 'marital_Status', 'IsGlobal',
    'risk_chance', 'age_group', 'is_peak_hour',
    'is_weekend', 'support'
]

num_cols = ['ResidentDay_log']

In [10]:
X_train ,X_test, y_train ,   y_test = train_test_split(
    X , y ,test_size = 0.2 , random_state = 42 , stratify = y
)

In [11]:
print("X_train.shape : "  ,X_train.shape)
print("y_train.shape : ",y_train.shape)
print("X_test.shape : ",X_test.shape)
print("y_test.shape : ",y_test.shape)

X_train.shape :  (12192, 13)
y_train.shape :  (12192,)
X_test.shape :  (3049, 13)
y_test.shape :  (3049,)


In [12]:

# cross flod validation 
cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [ ]:
# define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier( n_estimators=1000, random_state=42,class_weight="balanced" ),
    "XGboost":XGBClassifier(eval_metric='logloss', random_state=42),
     
}


TypeError: 'module' object is not callable

In [ ]:
from imblearn.over_sampling import SMOTE
results = []

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('smote', SMOTE(random_state=42)),
        ('scaler', StandardScaler()),
        ('model', model)
    ])

    scores = cross_val_score(
        pipeline,
        X_train,
        y_train,
        cv=cv,
        scoring='roc_auc'
    )

    results.append({
        "Model": name,
        "Mean ROC-AUC": scores.mean(),
        "Std": scores.std()
    })


In [ ]:
results_df = pd.DataFrame(results).sort_values(
    by="Mean ROC-AUC",
    ascending=False
)

results_df


,Model,Mean ROC-AUC,Std
2,XGboost,0.679654,0.003714
0,Logistic Regression,0.667405,0.003123
1,Random Forest,0.476398,0.004337
